### take the ICU clipped country shapes, buffer and convert from [-180 - 180] to [0 - 360]

In [1]:
%matplotlib inline

In [2]:
import os
import sys
import pathlib

In [3]:
import fiona
import geopandas as gpd

In [4]:
shapes_ipath = pathlib.Path.cwd().parents[1] / 'data' / 'shapefiles' / 'ICU' / 'clipped'

In [5]:
resources_path = pathlib.Path.cwd().parents[1] / 'resources'

In [6]:
shapes_opath = shapes_ipath / 'countries'

In [7]:
shapes_opath_converted = shapes_opath  / 'countries_converted'

In [8]:
shapefile_fname = 'Clipped_ICU_Countries2.shp'

In [9]:
shapefile = gpd.read_file(os.path.join(shapes_ipath, shapefile_fname))

In [10]:
def sanitize(country_name): 
    return country_name.replace(' ','_').replace(':','').replace('/','').replace('&','and')

### creates dictionnary mapping filenames to country names

In [11]:
dict_countries = {}

In [12]:
for country in shapefile.COUNTRYNAM.unique():
    dict_countries[sanitize(country)] = country

In [13]:
import json

with open(resources_path / 'dict_countries.json', 'w') as fp:
    json.dump(dict_countries, fp)

### loops and converts the shapes in the shapefile 

In [14]:
shapefile

,OBJECTID,COUNTRY_ID,COUNTRYNAM,ICU,Stress,SOPAC,FFA,SPC,SPREP,USP,TCSP,FORUM,ACP,area,Shape_Leng,Shape_Area,geometry
0,1,NC,New Caledonia,2,-6,0,0,0,0,0,0,0,0,0.000000e+00,30.331204,1.639128,"MULTIPOLYGON (((166.93318 -22.88216, 166.93263..."
1,2,PN,Pitcairn Islands,-3,-8,0,0,0,0,0,0,0,0,0.000000e+00,0.733043,0.004720,"MULTIPOLYGON (((-130.09494 -25.06890, -130.094..."
2,3,PG,Papua New Guinea,-3,5,0,0,0,0,0,0,0,0,0.000000e+00,229.361133,37.800908,"MULTIPOLYGON (((153.50899 -11.61978, 153.50839..."
3,4,SB,Solomon Islands,-9,-1,0,0,0,0,0,0,0,0,0.000000e+00,87.796755,2.341716,"MULTIPOLYGON (((168.82513 -12.28072, 168.82766..."
4,5,KI,Kiribati: Phoenix Islands,-5,-8,0,0,0,0,0,0,0,0,0.000000e+00,1.703490,0.003382,"MULTIPOLYGON (((-174.53215 -4.65909, -174.5274..."
5,6,FJ,Fiji,2,-2,0,0,0,0,0,0,0,0,0.000000e+00,42.655862,1.611568,"MULTIPOLYGON (((-178.84500 -21.04222, -178.845..."
6,7,TV,Tuvalu,-5,-8,0,0,0,0,0,0,0,0,0.000000e+00,2.220148,0.003414,"MULTIPOLYGON (((179.47405 -10.79220, 179.47329..."
7,8,CK,Northern Cook Islands,0,1,0,0,0,0,0,0,0,0,0.000000e+00,1.977260,0.002987,"MULTIPOLYGON (((-163.11351 -13.33375, -163.113..."
8,9,KI,Kiribati: Line Islands,-5,-14,0,0,0,0,0,0,0,0,0.000000e+00,5.140038,0.049134,"MULTIPOLYGON (((-151.81682 -11.42452, -151.816..."
9,10,FM,Federated States of Micronesia,4,3,0,0,0,0,0,0,0,0,0.000000e+00,11.676262,0.063027,"MULTIPOLYGON (((154.78083 1.02639, 154.77972 1..."


In [50]:
for country in shapefile.COUNTRYNAM.unique(): 
    country_fname = sanitize(country)
    shape_country = shapefile.query("COUNTRYNAM == '{}'".format(country))
    shape_country = shape_country.buffer(distance=0.1) 
    shape_country.to_file(os.path.join(shapes_opath, '{}'.format(country_fname))) 
    shapes = fiona.open(os.path.join(shapes_opath, '{}'.format(country_fname))) 
    if not os.path.exists(os.path.join(shapes_opath_converted, country_fname)): 
        os.mkdir(os.path.join(shapes_opath_converted, country_fname))
    with fiona.open(os.path.join(shapes_opath_converted, country_fname, 'shapes_{}.shp'.format(country_fname)), 'w', 'ESRI Shapefile', shapes.schema.copy(), crs = shapes.crs) as output:

        for shape in shapes:

    #         if shape['properties']['min_x'] < 0:

    #             shape['properties']['min_x'] += 360

    #         if shape['properties']['max_x'] < 0:

    #             shape['properties']['max_x'] += 360

            if (len(shape['geometry']['coordinates']) == 1):

                coords = shape['geometry']['coordinates']

                coords_mod = coords.copy()

                for j in range(len(coords[0])):

                    lon, lat = coords[0][j]

                    if lon < 0:

                        lon += 360

                    coords_mod[0][j] = (lon, lat)

                shape['geometry']['coordinates'] = coords_mod 

            else:

                for i in range(len(shape['geometry']['coordinates'])):

                    coords = shape['geometry']['coordinates'][i]

                    coords_mod = coords.copy()

                    for j in range(len(coords[0])):

                        lon, lat = coords[0][j]

                        if lon < 0:

                            lon += 360

                        coords_mod[0][j] = (lon, lat)

                    shape['geometry']['coordinates'][i] = coords_mod 

            output.write(shape)